<a href="https://colab.research.google.com/github/vmdvaldez/WeatherImageClassification/blob/master/Alexnet_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [158]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
!rm -rf /root/datasets
!mkdir /root/datasets

In [160]:
Archive('/content/drive/My Drive/Image.rar').extractall('/root/datasets')

In [161]:
!pip install split-folders

In [163]:
import split_folders
# Put them in a ratio of 80, 10, 10 in a folder called /root/datasets/Lab_3b_Gesture_Dataset_output
split_folders.ratio('/root/datasets/Image', output="/root/datasets/Image_output", seed=1337, ratio=(.7, .15, .15)) # default values

Copying files: 8408 files [00:01, 4834.88 files/s]


In [164]:
!pip3 install Pillow

In [165]:
!ln -s "/content/drive/My Drive/" "/bbbb"
!cp /bbbb/augment_images.sh /root
!cp /bbbb/data_augment.py /root

ln: failed to create symbolic link '/bbbb/My Drive': Operation not supported


In [ ]:
!bash /root/augment_images.sh

Augmenting Images in cloudyCreating Mirrored Images for /root/datasets/Image_output/train/cloudy/*
/root/datasets/Image_output/train/cloudy/37043582.jpg
/root/datasets/Image_output/train/cloudy/30051965.jpg
/root/datasets/Image_output/train/cloudy/378664.jpg
/root/datasets/Image_output/train/cloudy/13999112.jpg
/root/datasets/Image_output/train/cloudy/2053500.jpg
/root/datasets/Image_output/train/cloudy/42824132.jpg
/root/datasets/Image_output/train/cloudy/22257705.jpg
/root/datasets/Image_output/train/cloudy/11735675.jpg
/root/datasets/Image_output/train/cloudy/31745730.jpg
/root/datasets/Image_output/train/cloudy/9784699.jpg
/root/datasets/Image_output/train/cloudy/18494062.jpg
/root/datasets/Image_output/train/cloudy/59123690.jpg
/root/datasets/Image_output/train/cloudy/63431684.jpg
/root/datasets/Image_output/train/cloudy/45503402.jpg
/root/datasets/Image_output/train/cloudy/33045250.jpg
/root/datasets/Image_output/train/cloudy/32050444.jpg
/root/datasets/Image_output/train/cloudy/

In [9]:
import time
import os
import numpy as np
import torch

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

In [11]:
# define training and test data directories
data_dir = '/root/datasets/Image_output'
train_dir = os.path.join(data_dir, 'train/')
val_dir = os.path.join(data_dir, 'val/')
test_dir = os.path.join(data_dir, 'test/')

# classes are folders in each directory with these names
classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']

In [12]:
# load and transform data using ImageFolder

# resize all images to 224 x 224
data_transform = transforms.Compose([transforms.Resize((224,224)),
                                    transforms.ToTensor()])


train_data = datasets.ImageFolder(train_dir, transform=data_transform)
val_data = datasets.ImageFolder(val_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

# print out some data stats
print('Num training images: ', len(train_data))
print('Num validation images: ', len(val_data))
print('Num test images: ', len(test_data))

Num training images:  23541
Num validation images:  5044
Num test images:  5047


In [13]:
# alexnet
import torchvision.models

alexNet = torchvision.models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [14]:
# Save Features to Folder (assumes code from 1. has been evaluated)

import os
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

# location on Google Drive
train_path = '/content/drive/My Drive/Colab Notebooks/Project/Features5/Train'
val_path = '/content/drive/My Drive/Colab Notebooks/Project/Features5/Val'
test_path = '/content/drive/My Drive/Colab Notebooks/Project/Features5/Test'

# Prepare Dataloader (requires code from 1.)
batch_size = 1 # save 1 file at a time, hence batch_size = 1
num_workers = 1
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']

# save features to folder as tensors
n = 0
for img, label in train_loader:
  features = alexnet.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())

  folder_name = train_path + '/' + str(classes[label])
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  torch.save(features_tensor.squeeze(0), folder_name + '/' + str(n) + '.tensor')
  n += 1

# save features to folder as tensors
n = 0
for img, label in val_loader:
  features = alexnet.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())

  folder_name = val_path + '/' + str(classes[label])
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  torch.save(features_tensor.squeeze(0), folder_name + '/' + str(n) + '.tensor')
  n += 1

# save features to folder as tensors
n = 0
for img, label in test_loader:
  features = alexnet.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())

  folder_name = test_path + '/' + str(classes[label])
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  torch.save(features_tensor.squeeze(0), folder_name + '/' + str(n) + '.tensor')
  n += 1

In [18]:
# Load Tensor Files (features) from folder

# location on Google Drive
train_path = '/content/drive/My Drive/Colab Notebooks/Project/Features5/Train'
val_path = '/content/drive/My Drive/Colab Notebooks/Project/Features5/Val'
test_path = '/content/drive/My Drive/Colab Notebooks/Project/Features5/Test'

!ln -s "/content/drive/My Drive/Colab Notebooks/" "/aaaa"

!rm -rf /root/Train
!rm -rf /root/Val
!rm -rf /root/Test
!rm -rf /aaaa/Project/Features5

!cp -a /aaaa/Project/Features5 /root

train_path = "/root/Features5/Train"
val_path = "/root/Features5/Val"
test_path = "/root/Features5/Test"

#load features
# location on Google Drive
train_dataset = torchvision.datasets.DatasetFolder(train_path, loader=torch.load, extensions=('.tensor'))
val_dataset = torchvision.datasets.DatasetFolder(val_path, loader=torch.load, extensions=('.tensor'))
test_dataset = torchvision.datasets.DatasetFolder(test_path, loader=torch.load, extensions=('.tensor'))

# Prepare Dataloader
batch_size = 512
num_workers = 1
train_feature_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
val_feature_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_feature_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

# Verification Step - obtain one batch of features
dataiter = iter(train_feature_loader)
features, labels = dataiter.next()
print(features.shape)
print(labels.shape)

ln: failed to create symbolic link '/aaaa/Colab Notebooks': Operation not supported
cp: cannot stat '/aaaa/Project/Features5': No such file or directory
torch.Size([512, 256, 6, 6])
torch.Size([512])


In [19]:
def get_accuracy(model, train=False):
    if train:
        data_loader = train_feature_loader
    else:
        data_loader = val_feature_loader

    correct = 0
    total = 0
    for imgs, labels in data_loader:
        
         
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [43]:
def get_accuracy(model, data_loader, conf_matrix=False):
    # calculate total correct predictions within the specified data_set

    
#     classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny', 'z-other']
    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    x = np.zeros(shape=(len(classes), len(classes)))
#     print(x.shape)

    l = 0
    correct, total = 0 , 0
    for images, labels in data_loader:
        
#         if l > 10000:
#             break
        
        images = images.cuda()
        labels = labels.cuda()
        
        outputs = model(images)
        
        pred = outputs.max(1, keepdim=True)[1]
        
        if conf_matrix:
            i = labels.view_as(pred).item()
            j = pred.item()
            x[i][j] += 1
            
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += images.shape[0]
        
        l+=1
    if conf_matrix:
        return (correct / total), x
    else:
        return (correct / total)

In [21]:
# For getting Validation Loss

def evaluate(net, loader, criterion):
    total_loss = 0.0
    for i, data in enumerate(loader, 0):
        inputs = data
        outputs = net(inputs)
        loss = criterion(outputs, labels.float())
        total_loss += loss.item()
    loss = float(total_loss) / (i + 1)
    return loss

In [22]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [155]:
def train2(model, train_loader, val_loader, batch_size=1024, num_epochs=1):
    #train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    # train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           #num_workers=num_workers, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)

    iters, losses, train_acc, val_acc = [], [], [], []

    # training
    n = 0 # the number of iterations
    start_time=time.time()
    for epoch in range(num_epochs):
        mini_b=0
        mini_batch_correct = 0
        Mini_batch_total = 0
        for imgs, labels in iter(train_loader):
          
            
            #############################################
            #To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            #############################################

          #### ALNC is alexNet.features (AlexNet without classifier) ####
            out = model(imgs)             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch



            ##### Mini_batch Accuracy ##### We don't compute accuracy on the whole trainig set in every iteration!
            pred = out.max(1, keepdim=True)[1]
            mini_batch_correct = pred.eq(labels.view_as(pred)).sum().item()
            Mini_batch_total = imgs.shape[0]
            train_acc.append((mini_batch_correct / Mini_batch_total))
           ###########################

          # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            val_acc.append(get_accuracy(model, val_loader))  # compute validation accuracy
            #val_loss.append(evaluate(model, val_loader, criterion))
            n += 1
            mini_b += 1
            print("Iteration: ",n,'Progress: % 6.2f ' % ((epoch * len(train_loader) + mini_b) / (num_epochs * len(train_loader))*100),'%', "Time Elapsed: % 6.2f s " % (time.time()-start_time))
        # Save the current model (checkpoint) to a file
        model_path = get_model_name("CNNClassifier5", batch_size, 0.015, epoch)
        new_path = "/content/drive/My Drive/Project_Alex_3/" + model_path 
        torch.save(model.state_dict(), new_path)
        torch.save(model.state_dict(), model_path)
            
        print("Epoch %d Finished. " % epoch ,"Time per Epoch: % 6.2f s "% ((time.time()-start_time) / (epoch +1)))
        print(epoch, train_acc[-1], val_acc[-1])


    end_time= time.time()
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    #plt.plot(iters, val_loss, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Training")
    plt.plot(iters, val_acc, label="Validation")    
    plt.xlabel("Iterations")
    plt.ylabel("Validation Accuracy")
    plt.legend(loc='best')
    plt.show()

    train_acc.append(get_accuracy(model, train_loader))
    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    print ("Total time:  % 6.2f s  Time per Epoch: % 6.2f s " % ( (end_time-start_time), ((end_time-start_time) / num_epochs) ))
    

In [141]:
torch.manual_seed(1) # set the random seed

class CNNClassifier5(nn.Module):
    def __init__(self):
        super(CNNClassifier5, self).__init__()
        self.conv1 = nn.Conv2d(256, 5, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(5*2*2, 64)
        self.fc2 = nn.Linear(64, 9)

    def forward(self, img):
        x = self.pool(F.relu(self.conv1(img)))
        x = x.view(-1, 5*2*2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [156]:
use_cuda = True

model =CNNClassifier5()
ALNC = alexNet.features

if use_cuda and torch.cuda.is_available():
  ALNC.cuda()
  model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

# Prepare Dataloader
batch_size = 1024
num_workers = 1
train_feature_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
val_feature_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_feature_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

#proper model
train2(model, train_feature_loader, val_feature_loader, num_epochs=50)

CUDA is available!  Training on GPU ...
Iteration:  1 Progress:   0.09  % Time Elapsed:   1.52 s 
Iteration:  2 Progress:   0.17  % Time Elapsed:   2.75 s 
Iteration:  3 Progress:   0.26  % Time Elapsed:   3.94 s 
Iteration:  4 Progress:   0.35  % Time Elapsed:   5.18 s 
Iteration:  5 Progress:   0.43  % Time Elapsed:   6.38 s 
Iteration:  6 Progress:   0.52  % Time Elapsed:   7.62 s 
Iteration:  7 Progress:   0.61  % Time Elapsed:   8.84 s 
Iteration:  8 Progress:   0.70  % Time Elapsed:  10.13 s 
Iteration:  9 Progress:   0.78  % Time Elapsed:  11.45 s 
Iteration:  10 Progress:   0.87  % Time Elapsed:  12.64 s 
Iteration:  11 Progress:   0.96  % Time Elapsed:  13.90 s 
Iteration:  12 Progress:   1.04  % Time Elapsed:  15.10 s 
Iteration:  13 Progress:   1.13  % Time Elapsed:  16.34 s 
Iteration:  14 Progress:   1.22  % Time Elapsed:  17.49 s 
Iteration:  15 Progress:   1.30  % Time Elapsed:  18.68 s 
Iteration:  16 Progress:   1.39  % Time Elapsed:  19.88 s 
Iteration:  17 Progress: 

KeyboardInterrupt: ignored

In [144]:
def get_accuracy(model, data_loader, conf_matrix=False):
    # calculate total correct predictions within the specified data_set

    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    x = np.zeros(shape=(len(classes), len(classes)))

    l = 0
    correct, total = 0 , 0
    for images, labels in data_loader:
        
        images = images.cuda()
        labels = labels.cuda()
        
        outputs = model(images)
        
        pred = outputs.max(1, keepdim=True)[1]
        
        if conf_matrix:
            for k in range(labels.cpu().shape[0]):
              i = labels.view_as(pred)[k]
              j = pred.cpu()[k]
              x[i][j] += 1
            
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += images.shape[0]
        
        l+=1
    if conf_matrix:
        return (correct / total), x
    else:
        return (correct / total)

In [145]:
def print_conf_matrix(conf_matrix):
    print("\n\n")

    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']

    for i in range(len(classes)):
        if i == 0:
            print('', end='\t')

        print(classes[i], end = '\t')

    print('\n')

    for i in range(len(classes)):

        print(classes[i], end ='\t')
        for j in range(len(classes)):
            print(conf_matrix[i][j], end='\t')
        print('\n')

In [153]:
x, y = get_accuracy(model, val_feature_loader, conf_matrix=True)
print(x)
print_conf_matrix(y)

0.7848929421094369



	cloudy	foggy	rain	snow	sunny	

cloudy	671.0	56.0	212.0	33.0	123.0	

foggy	21.0	782.0	41.0	24.0	11.0	

rain	59.0	36.0	832.0	63.0	22.0	

snow	26.0	35.0	98.0	849.0	27.0	

sunny	103.0	19.0	60.0	16.0	825.0	



In [152]:
x, y = get_accuracy(model, train_feature_loader, conf_matrix=True)
print(x)
print_conf_matrix(y)

0.8754088611358906



	cloudy	foggy	rain	snow	sunny	

cloudy	3645.0	194.0	768.0	92.0	413.0	

foggy	32.0	3886.0	105.0	54.0	25.0	

rain	127.0	72.0	4381.0	109.0	34.0	

snow	61.0	88.0	265.0	4352.0	64.0	

sunny	215.0	72.0	109.0	34.0	4344.0	



In [151]:
x, y = get_accuracy(model, test_feature_loader, conf_matrix=True)
print(x)
print_conf_matrix(y)

0.7790766792153755



	cloudy	foggy	rain	snow	sunny	

cloudy	638.0	68.0	208.0	35.0	148.0	

foggy	31.0	777.0	44.0	17.0	10.0	

rain	62.0	33.0	845.0	49.0	24.0	

snow	22.0	31.0	90.0	858.0	34.0	

sunny	100.0	26.0	60.0	23.0	814.0	

